In [1]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

Saving 202501.xlsx to 202501.xlsx


In [2]:
excel_path = '202501.xlsx'
df = pd.read_excel(excel_path)

In [13]:
# 금속 컬럼 추출
column_pattern = ['(ng/m3)', '(ug/m3)']
columns_to_analyze = [c for c in df.columns if any(p in c for p in column_pattern)]

In [14]:
# 최빈값 계산, 여러 개면 가장 작은 값 반환
def safe_mode(series):
    s = series.dropna() # Nan 값 제거
    if s.empty:
        return np.nan
    modes = s.mode() # 최빈값 구하기
    return modes.min() if not modes.empty else np.nan

In [15]:
# 백분위 계산
def nanpercentile(series, q):
    arr = series.dropna().to_numpy() # Nan 값 제거
    if arr.size == 0:
        return np.nan
    return float(np.percentile(arr, q, method='linear'))

In [16]:
# 통계량 계산
def compute_stats(s):
    x = s.dropna() # Nan값 제거
    n = len(x) # 유효 데이터 개수
    if n == 0: # 값이 없으면
        return dict.fromkeys(['Mean','Std. Error of Mean','Mode','Median',
                              'Std. Deviation','Range','Minimum','Maximum',
                              'Sum','P5','P25','P50','P75','P95','P100'], np.nan)

    mean = x.mean() # 평균
    std = x.std(ddof=1) # 표본 표준편차
    sem = std / np.sqrt(n) # 평균의 표준오차
    xmin, xmax = x.min(), x.max() # 최소값, 최대값

    return {
        'Mean': mean,
        'Std. Error of Mean': sem,
        'Mode': safe_mode(x), # 최빈값
        'Median': x.median(), # 중앙값
        'Std. Deviation': std, # 표준편차
        'Range': xmax - xmin, # 범위
        'Minimum': xmin,
        'Maximum': xmax,
        'Sum': x.sum(),
        'P5': nanpercentile(x, 5), # 5% 백분위수
        'P25': nanpercentile(x, 25), # 25% 백분위수
        'P50': nanpercentile(x, 50), # 50% 백분위수
        'P75': nanpercentile(x, 75), # 75% 백분위수
        'P95': nanpercentile(x, 95), # 95% 백분위수
        'P100': nanpercentile(x, 100) # 100% 백분위수
    }

In [17]:
# 모든 대상 컬럼에 대해 통계 계산
stats_dict = {col: compute_stats(df[col]) for col in columns_to_analyze}

# 표 형태로 변환
stats_table = pd.DataFrame(stats_dict)

# 소수점 자리수 조정
stats_table_rounded = stats_table.round(3)

# 표 출력
from IPython.display import display
display(stats_table_rounded)

,Conc(ug/m3),Al(ng/m3),Si(ng/m3),S(ng/m3),K(ng/m3),Ca(ng/m3),Ti(ng/m3),V(ng/m3),Cr(ng/m3),Mn(ng/m3),...,Cd(ng/m3),In(ng/m3),Sn(ng/m3),Sb(ng/m3),Te(ng/m3),Cs(ng/m3),Ba(ng/m3),Ce(ng/m3),Pb(ng/m3),Bi(ng/m3)
Mean,16.865,535.975,1165.064,4073.718,311.863,143.559,0.816,3.027,2.339,5.757,...,215.833,26.379,5.930,1.957,0.974,1.065,2.222,3.992,9.519,2.899
Std. Error of Mean,0.484,9.582,40.235,71.223,6.220,2.899,0.144,0.078,0.093,0.186,...,2.284,0.843,0.601,0.952,0.860,0.241,0.315,0.327,0.467,0.124
Mode,13.720,166.260,903.080,4337.730,206.350,96.000,0.880,3.140,3.290,0.230,...,135.750,-5.220,-4.550,-29.110,-32.660,-2.300,0.000,-1.130,1.390,3.780
Median,13.720,510.645,943.235,3633.105,285.530,132.650,0.460,2.850,2.545,6.570,...,211.845,24.610,5.080,1.410,0.285,-0.600,0.000,1.680,6.290,2.935
Std. Deviation,11.012,215.108,903.284,1598.947,139.649,65.078,3.225,1.756,2.088,4.171,...,51.286,18.916,13.495,21.364,19.305,5.406,7.065,7.342,10.473,2.782
Range,96.190,1756.450,8911.150,9116.300,943.610,637.760,31.480,11.000,10.610,19.020,...,267.440,129.320,78.240,123.470,119.320,36.640,63.890,53.920,59.750,15.760
Minimum,-48.570,166.260,594.850,1094.290,122.710,78.880,-3.520,-0.110,-1.940,-2.320,...,111.090,-20.740,-29.000,-56.850,-55.130,-4.510,0.000,-2.390,-2.860,-3.530
Maximum,47.620,1922.710,9506.000,10210.590,1066.320,716.640,27.960,10.890,8.670,16.700,...,378.530,108.580,49.240,66.620,64.190,32.130,63.890,51.530,56.890,12.230
Sum,8719.460,270131.320,587192.150,2053153.730,157178.790,72353.610,411.470,1525.810,1178.990,2901.290,...,108779.870,13294.870,2988.760,986.310,490.710,537.000,1119.690,2011.930,4797.490,1460.930
P5,4.992,275.793,656.102,2212.358,146.000,94.458,-2.906,0.479,-0.838,-0.887,...,144.451,0.127,-15.216,-32.040,-29.664,-3.177,0.000,-1.580,-0.926,-1.558
